In [0]:
#https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_classification_rnn.ipynb#scrollTo=z682XYsrjkY9
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
!pip install tf-nightly
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Conv1D, MaxPooling1D, GRU

TensorFlow 2.x selected.


In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [0]:
dataset, info = tfds.load('lm1b/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Buffered data was truncated after reaching the output size limit.

In [0]:
model = Sequential([
    Embedding(encoder.vocab_size, 32, mask_zero=True),
    Conv1D(32, 3, padding='same', activation = 'relu'),
    MaxPooling1D(2),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [0]:
model.fit(train_dataset, epochs=5,
                      steps_per_epoch=32,
                      validation_data=test_dataset,
                      validation_steps=700
                      )

test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [0]:
plot_graphs(history, 'loss')

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
predictions = model.predict('In chacma baboons, male-infant relationships can be linked to both formation of friendships and paternity success [30,31].')
print(predictions)